In [24]:
import matplotlib.pyplot as plt
%matplotlib widget
from ipywidgets import *
import numpy as np
import sys

In [25]:
import taurex.log
taurex.log.disableLogging()

In [26]:
from taurex.cache import OpacityCache,CIACache
OpacityCache().clear_cache()
OpacityCache().set_opacity_path("../atmosphere/xsecs")
CIACache().set_cia_path("../atmosphere/cia/hitran")


In [27]:
from taurex.temperature import Guillot2010
guillot = Guillot2010(T_irr=1200.0)

In [28]:
from taurex.planet import Planet
planet = Planet(planet_radius=1.7420,planet_mass=1.170)  #in jupiter masses

#parameters retrieved from https://ui.adsabs.harvard.edu/abs/2024AJ....168..231S/abstract

In [29]:
from taurex.stellar import BlackbodyStar

star = BlackbodyStar(temperature=6628,radius=1.461)

In [30]:
from taurex.chemistry import TaurexChemistry
from taurex.chemistry import ConstantGas
chemistry = TaurexChemistry(fill_gases=['H2','He'],ratio=0.172)
# H2O, CH4, CO2 and CO randomised within [10^{-8},10^{-2}].
rng = np.random.default_rng(seed=100)
chemistry.addGas(ConstantGas('H20',mix_ratio=rng.uniform(1e-8,1e-2)))
chemistry.addGas(ConstantGas('CH4',mix_ratio=rng.uniform(1e-8,1e-2)))
chemistry.addGas(ConstantGas('CO2',mix_ratio=rng.uniform(1e-8,1e-2)))
chemistry.addGas(ConstantGas('CO',mix_ratio=rng.uniform(1e-8,1e-2)))

In [31]:
from taurex.model import TransmissionModel
from taurex.contributions import AbsorptionContribution
from taurex.contributions import CIAContribution
from taurex.contributions import RayleighContribution

tm = TransmissionModel(planet=planet,
                       temperature_profile=guillot,
                       chemistry=chemistry,
                       star=star,
                       atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)
tm.add_contribution(AbsorptionContribution())
tm.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
tm.add_contribution(RayleighContribution())
tm.build()
res = tm.model()
res

InvalidModelException: No active absorbing molecules